<a href="https://colab.research.google.com/github/hou-rai3/PG1_07_task7/blob/main/07_%E8%AA%B2%E9%A1%8C7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import time
import pygame
import pygame.locals
import sys

# オーバーシュート防止用のclamp関数
def clamp(value, min_val, max_val):
    return max(min(value, max_val), min_val)

# 角度を-180°～180°にとどめる関数
def normalize_angle(angle):
    while angle > 180:
        angle -= 360
    while angle < -180:
        angle += 360
    return angle

# PID計算機（角度）
def pid_angle_calculation(target, current, kp, ki, kd, integral, pre_error, dt):
    error = normalize_angle(target - current)  # 角度差を正規化
    integral += error * dt  # 積分値を更新
    deriv = (error - pre_error) / dt  # 微分値を計算
    pid_output = kp * error + ki * integral + kd * deriv  # PID出力を計算
    pid_output = max(min(pid_output, 10.0), -10.0)
    current = normalize_angle(current + pid_output)  # 現在の角度を更新
    return current, integral, error

# PID計算機（速度）
def pid_calculation(target, current, kp, ki, kd, integral, pre_error, dt):
    error = target - current  # 誤差を計算
    integral += error * dt  # 積分値を更新
    deriv = (error - pre_error) / dt  # 微分値を計算
    pid_output = kp * error + ki * integral + kd * deriv  # PID出力を計算
    pid_output = max(min(pid_output, 10.0), -10.0)
    current += pid_output  # 現在の速度を更新
    return current, integral, error

# Pygame初期化
pygame.init()
pygame.display.set_mode((640, 480))
pygame.display.set_caption("Independent Steering Simulation")
surface = pygame.display.get_surface()
surface.fill((0, 128, 0))

font = pygame.font.SysFont(None, 24)

# タイヤの初期設定
x, y = 320.0, 240.0  # 真ん中スタートにする
left_wheel_angle = 0.0  # ステア角
left_wheel_speed = 0.0  # タイヤの速度
target_x, target_y = x, y  # 初期目標位置

# ゲイン
kp_angle, ki_angle, kd_angle = 0.07, 0.00, 0.001
kp_speed, ki_speed, kd_speed = 0.19, 0.00, 0.00
integral_angle, integral_speed = 0.0, 0.0
pre_error_angle, pre_error_speed = 0.0, 0.0

clock = pygame.time.Clock()

while True:
    dt = clock.tick(60) / 1000.0  # フレーム間の時間を秒単位で計算
    for event in pygame.event.get():
        if event.type == pygame.locals.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.locals.MOUSEMOTION:
            target_x, target_y = event.pos

    # ベクトル計算で目標ステア角度を決定
    dx = target_x - x
    dy = target_y - y
    target_angle = math.degrees(math.atan2(dy, dx))  # ラジアンを度に変換
    target_speed = math.sqrt(dx**2 + dy**2) / 30.0  # 距離に基づいた目標速度

    # PID制御でタイヤのステア角度と速度を計算
    left_wheel_angle, integral_angle, pre_error_angle = pid_angle_calculation(
        target_angle, left_wheel_angle, kp_angle, ki_angle, kd_angle, integral_angle, pre_error_angle, dt)
    left_wheel_speed, integral_speed, pre_error_speed = pid_calculation(
        target_speed, left_wheel_speed, kp_speed, ki_speed, kd_speed, integral_speed, pre_error_speed, dt)

    # 現在の位置を速度に基づいて更新
    x += math.cos(math.radians(left_wheel_angle)) * left_wheel_speed
    y += math.sin(math.radians(left_wheel_angle)) * left_wheel_speed

    # 画面描画
    # タイヤの相対座標
    wheel_offsets = [(-30, -30), (-30, 30), (30, -30), (30, 30)]

    # 画面描画
    surface.fill((0, 128, 0))

    # 車体中央とタイヤを描画
    for offset_x, offset_y in wheel_offsets:
        tire_x = x + offset_x
        tire_y = y + offset_y
        pygame.draw.circle(surface, (255, 255, 255),
                           (int(tire_x), int(tire_y)), 10)  # タイヤの描画

        # 各タイヤから赤い線を描画
        line_end_x = tire_x + math.cos(math.radians(left_wheel_angle)) * 30
        line_end_y = tire_y + math.sin(math.radians(left_wheel_angle)) * 30
        pygame.draw.line(surface, (255, 0, 0), (int(tire_x), int(
            tire_y)), (int(line_end_x), int(line_end_y)), 3)

    # デバッグ情報を表示
    debug_info = [
        f"Target: ({target_x:.2f}, {target_y:.2f})",
        f"Position: ({x:.2f}, {y:.2f})",
        f"Target Angle: {target_angle:.2f}°",
        f"Wheel Angle: {left_wheel_angle:.2f}°",
        f"Target Speed: {target_speed:.2f}",
        f"Wheel Speed: {left_wheel_speed:.2f}",
    ]
    for i, text in enumerate(debug_info):
        debug_surface = font.render(text, True, (255, 255, 255))
        surface.blit(debug_surface, (10, 10 + i * 20))

    pygame.display.update()